In [1]:
%load_ext autotime

In [2]:
# Importar librerias
import pandas as pd
import numpy as np

import zipfile
from datetime import date
from sklearn import metrics
from sklearn.metrics import accuracy_score


from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

from numpy.testing import assert_allclose
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta

import os
from pathlib import Path


Using TensorFlow backend.


time: 9.85 s


In [3]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

time: 4 ms


In [4]:
exp_name = '7'
folder = 'Resultados/' + exp_name
my_file = Path(folder)
if os.path.exists(my_file):
    print('already exists')
else:
    os.makedirs(folder)
    


time: 154 ms


In [5]:
ch_folder = folder + '/Checkpoints'
my_file = Path(ch_folder)
if os.path.exists(my_file):
    print('already exists')
else:
    os.makedirs(ch_folder)

time: 340 ms


### Read datasets

In [6]:
#Importar los datasets
url_embeddings_average_individual = zipfile.ZipFile('../Data/embeddings_average_individual.zip')
url_embeddings_sum_individual = zipfile.ZipFile('../Data/embeddings_sum_individual.zip')

embeddings_average_individual = pd.read_csv(url_embeddings_average_individual.open('embeddings_average_individual.csv'), index_col = 0)
embeddings_sum_individual =pd.read_csv(url_embeddings_sum_individual.open('embeddings_sum_individual.csv'), index_col = 0)

embeddings_average_individual['Date'] =  pd.to_datetime(embeddings_average_individual['Date'], format='%Y-%m-%d')
# embeddings_average_individual.sort_values('Date', inplace=True)
# embeddings_average_individual.reset_index(drop = True , inplace=True)

embeddings_average_individual.tail(2)

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,Label,Date,Top
49716,-0.082072,0.099915,-0.015503,0.115560,-0.072611,0.070435,0.042613,-0.041026,0.012126,-0.020508,...,-0.062215,-0.078471,0.046008,0.005717,-0.071452,0.122559,0.07622,0,2008-08-08,24
49717,-0.064514,0.013916,-0.028976,0.058716,-0.078369,-0.057312,-0.077515,-0.234467,0.050751,0.020508,...,-0.010498,0.081284,0.040283,-0.108978,0.033783,0.028870,0.03418,0,2008-08-08,25


time: 14.4 s


### Embedding Promedio

In [7]:
# Selecciono la fecha para la cual hago el corte de train y test
training_end = pd.to_datetime("2013-12-31")
num_training = len(embeddings_average_individual[(embeddings_average_individual["Date"]) <= training_end])

time: 69 ms


In [8]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_average_individual[embeddings_average_individual['Date']<='2014-12-31']

time: 245 ms


In [9]:
# Se separa en train y test
x_train = data.drop(["Top","Label", "Date"], axis=1)[:num_training]
x_test = data.drop(["Top",'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]


x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(len(x_train), 1, 300)
reshape_x_train.shape

x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(len(x_test), 1, 300)
reshape_x_test.shape

(6300, 1, 300)

time: 211 ms


### Definir espacio de busqueda

In [10]:
space = {
    'units1': hp.choice('units1', [10, 64, 128, 256, 512]),
    'units2': hp.choice('units2', [10, 64, 128, 256, 512]),
                 
    'dropout1': hp.choice('dropout1', [0.2,0.3,0.1]),
    
    'batch_size' : hp.choice('batch_size', [128,256,512]),
    'nb_epochs' : hp.choice('nb_epochs', [50]),

    'optimizer':  hp.choice('optimizer', [ 'adam','adadelta']),   
    'activation': 'relu'    
    }

time: 185 ms


### Definir busqueda bayesiana

In [11]:
#Objective function that hyperopt will minimize
from timeit import default_timer as timer

def objective(params):
    
    import ml_metrics

    
    start = timer()
    print ('Params testing: ', params)
    print ('\n ')
    
    model = Sequential()
    model.add(LSTM(params['units1'], input_shape=(1,300), return_sequences=True))
    model.add(Dropout(params['dropout1']))
    model.add(LSTM(params['units2'], return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='mean_squared_error', optimizer='adam')
    # compile the model
    model.compile(optimizer=params['optimizer'], loss='binary_crossentropy', metrics=['accuracy'])


    
    #includes the call back object
    model.fit(reshape_x_train, y_train, epochs=params['nb_epochs'], batch_size=params['batch_size'],
              verbose = 0, validation_data=(reshape_x_test, y_test))
     
    #predict the test set 
    score, acc = model.evaluate(reshape_x_test, y_test, verbose=0)
    
    run_time = timer() - start
    
    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([-acc, params, score, run_time])
    of_connection.close()
    
    
    print('Test accuracy:', acc)
 
    return {'loss': -acc, 'status': STATUS_OK, 'train_time': run_time,}



time: 112 ms


### Almacenar resultados de cada iteración

In [12]:
from hyperopt import tpe

tpe_algorithm = tpe.suggest

time: 112 ms


In [13]:
import csv

from hyperopt import Trials

bayes_trials = Trials()

# File to save first results
out_file = folder + '/gbm_results.csv'
of_connection = open(out_file, 'w')

writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'score','time'])
of_connection.close()

time: 158 ms


### Lanzar optimización

In [14]:
trials = Trials()


# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = 10, trials = bayes_trials,
            verbose = 1, rstate= np.random.RandomState(50))


Params testing:                                                                                                        
{'activation': 'relu', 'batch_size': 512, 'dropout1': 0.3, 'nb_epochs': 50, 'optimizer': 'adadelta', 'units1': 128, 'units2': 128}
Test accuracy:                                                                                                         
0.4920634925365448                                                                                                     
Params testing:                                                                                                        
{'activation': 'relu', 'batch_size': 256, 'dropout1': 0.3, 'nb_epochs': 50, 'optimizer': 'adam', 'units1': 64, 'units2': 10}
Test accuracy:                                                                                                         
0.5046031475067139                                                                                                     
Params testing:         

### Exportar bayesiana, por si quisiera retomar donde queda

In [15]:
import pickle
pickle.dump(bayes_trials, open(folder + '/trials.pkl', 'wb'))

time: 6 ms


### Leer mejores parametros

In [16]:
# Sort the trials with lowest loss (highest AUC) first
bayes_trials_results  = sorted(bayes_trials.results, key = lambda x: x['loss'])
bayes_trials_results [:2]

[{'loss': -0.5060317516326904,
  'status': 'ok',
  'train_time': 384.75509810000005},
 {'loss': -0.5046031475067139, 'status': 'ok', 'train_time': 57.4594458}]

time: 149 ms


In [17]:
results = pd.read_csv(folder + '/gbm_results.csv')

# Sort with best scores on top and reset index for slicing
results.sort_values('loss', ascending = True, inplace = True)
results.reset_index(inplace = True, drop = True)
results.head()

,loss,params,score,time
0,-0.506032,"{'activation': 'relu', 'batch_size': 256, 'dro...",NaN,384.755098
1,-0.504603,"{'activation': 'relu', 'batch_size': 256, 'dro...",NaN,57.459446
2,-0.498730,"{'activation': 'relu', 'batch_size': 256, 'dro...",NaN,539.432155
3,-0.497302,"{'activation': 'relu', 'batch_size': 256, 'dro...",NaN,257.556877
4,-0.497143,"{'activation': 'relu', 'batch_size': 256, 'dro...",NaN,517.148081


time: 152 ms


In [18]:
import ast

# Convert from a string to a dictionary
ast.literal_eval(results.loc[0, 'params'])

{'activation': 'relu',
 'batch_size': 256,
 'dropout1': 0.3,
 'nb_epochs': 50,
 'optimizer': 'adam',
 'units1': 10,
 'units2': 512}

time: 104 ms


In [19]:
# Extract the ideal number of estimators and hyperparameters
best_bayes_params = ast.literal_eval(results.loc[0, 'params']).copy()
best_bayes_params

{'activation': 'relu',
 'batch_size': 256,
 'dropout1': 0.3,
 'nb_epochs': 50,
 'optimizer': 'adam',
 'units1': 10,
 'units2': 512}

time: 121 ms


### Definir datasets de testeo

In [20]:
# Selecciono la fecha para la cual hago el corte de train y test
training_end = pd.to_datetime("2014-12-31")
num_training = len(embeddings_average_individual[(embeddings_average_individual["Date"]) <= training_end])

time: 178 ms


In [21]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_average_individual

time: 90.5 ms


In [22]:
# Se separa en train y test
x_train = data.drop(["Top","Label", "Date"], axis=1)[:num_training]
x_test = data.drop(["Top",'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]


x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(len(x_train), 1, 300)
reshape_x_train.shape

x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(len(x_test), 1, 300)
reshape_x_test.shape

(9450, 1, 300)

time: 314 ms


In [23]:
model = Sequential()
model.add(LSTM(best_bayes_params['units1'], input_shape=(1,300), return_sequences=True))
model.add(Dropout(best_bayes_params['dropout1']))
model.add(LSTM(best_bayes_params['units2'], return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='mean_squared_error', optimizer='adam')
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


#model.compile(loss='mean_squared_error', optimizer='adam')
# compile the model
model.compile(loss='binary_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])


# define the checkpoint
filepath= ch_folder + "/word2vec-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

time: 1.16 s


In [24]:
# fit the model
model.fit(reshape_x_train, y_train,
          epochs=best_bayes_params['nb_epochs'], 
          batch_size=best_bayes_params['batch_size'], callbacks=callbacks_list)


model.save(folder + '/keras_model.h5')
model.summary()

Epoch 1/50
40268/40268 [==============================] - ETA: 2:10 - loss: 0.6931 - accuracy: 0.57 - ETA: 1:10 - loss: 0.6929 - accuracy: 0.56 - ETA: 50s - loss: 0.6930 - accuracy: 0.5443 - ETA: 40s - loss: 0.6928 - accuracy: 0.543 - ETA: 34s - loss: 0.6926 - accuracy: 0.545 - ETA: 30s - loss: 0.6922 - accuracy: 0.551 - ETA: 27s - loss: 0.6922 - accuracy: 0.547 - ETA: 25s - loss: 0.6924 - accuracy: 0.540 - ETA: 23s - loss: 0.6924 - accuracy: 0.537 - ETA: 21s - loss: 0.6918 - accuracy: 0.544 - ETA: 20s - loss: 0.6917 - accuracy: 0.545 - ETA: 19s - loss: 0.6914 - accuracy: 0.546 - ETA: 19s - loss: 0.6914 - accuracy: 0.545 - ETA: 18s - loss: 0.6918 - accuracy: 0.540 - ETA: 17s - loss: 0.6918 - accuracy: 0.539 - ETA: 17s - loss: 0.6914 - accuracy: 0.542 - ETA: 16s - loss: 0.6911 - accuracy: 0.544 - ETA: 16s - loss: 0.6909 - accuracy: 0.544 - ETA: 15s - loss: 0.6907 - accuracy: 0.545 - ETA: 15s - loss: 0.6907 - accuracy: 0.545 - ETA: 14s - loss: 0.6905 - accuracy: 0.545 - ETA: 14s - loss: 

40268/40268 [==============================] - ETA: 10s - loss: 0.6865 - accuracy: 0.558 - ETA: 10s - loss: 0.6867 - accuracy: 0.558 - ETA: 10s - loss: 0.6867 - accuracy: 0.558 - ETA: 10s - loss: 0.6877 - accuracy: 0.552 - ETA: 10s - loss: 0.6889 - accuracy: 0.545 - ETA: 10s - loss: 0.6884 - accuracy: 0.548 - ETA: 10s - loss: 0.6890 - accuracy: 0.544 - ETA: 10s - loss: 0.6890 - accuracy: 0.544 - ETA: 10s - loss: 0.6890 - accuracy: 0.545 - ETA: 10s - loss: 0.6892 - accuracy: 0.543 - ETA: 9s - loss: 0.6898 - accuracy: 0.540 - ETA: 9s - loss: 0.6902 - accuracy: 0.53 - ETA: 9s - loss: 0.6896 - accuracy: 0.54 - ETA: 9s - loss: 0.6898 - accuracy: 0.54 - ETA: 9s - loss: 0.6896 - accuracy: 0.54 - ETA: 9s - loss: 0.6886 - accuracy: 0.54 - ETA: 9s - loss: 0.6889 - accuracy: 0.54 - ETA: 9s - loss: 0.6881 - accuracy: 0.55 - ETA: 9s - loss: 0.6884 - accuracy: 0.54 - ETA: 9s - loss: 0.6885 - accuracy: 0.54 - ETA: 9s - loss: 0.6885 - accuracy: 0.54 - ETA: 9s - loss: 0.6886 - accuracy: 0.54 - ETA: 9s 

40268/40268 [==============================] - ETA: 9s - loss: 0.6859 - accuracy: 0.56 - ETA: 10s - loss: 0.6865 - accuracy: 0.560 - ETA: 10s - loss: 0.6878 - accuracy: 0.553 - ETA: 10s - loss: 0.6868 - accuracy: 0.558 - ETA: 10s - loss: 0.6881 - accuracy: 0.551 - ETA: 10s - loss: 0.6867 - accuracy: 0.560 - ETA: 10s - loss: 0.6880 - accuracy: 0.552 - ETA: 10s - loss: 0.6881 - accuracy: 0.550 - ETA: 10s - loss: 0.6883 - accuracy: 0.550 - ETA: 10s - loss: 0.6881 - accuracy: 0.551 - ETA: 10s - loss: 0.6883 - accuracy: 0.549 - ETA: 10s - loss: 0.6885 - accuracy: 0.547 - ETA: 10s - loss: 0.6885 - accuracy: 0.547 - ETA: 10s - loss: 0.6884 - accuracy: 0.548 - ETA: 9s - loss: 0.6880 - accuracy: 0.550 - ETA: 9s - loss: 0.6878 - accuracy: 0.55 - ETA: 9s - loss: 0.6878 - accuracy: 0.55 - ETA: 9s - loss: 0.6881 - accuracy: 0.54 - ETA: 9s - loss: 0.6881 - accuracy: 0.54 - ETA: 9s - loss: 0.6882 - accuracy: 0.54 - ETA: 9s - loss: 0.6884 - accuracy: 0.54 - ETA: 9s - loss: 0.6885 - accuracy: 0.54 - ET

40268/40268 [==============================] - ETA: 9s - loss: 0.6932 - accuracy: 0.51 - ETA: 9s - loss: 0.6896 - accuracy: 0.54 - ETA: 10s - loss: 0.6891 - accuracy: 0.546 - ETA: 10s - loss: 0.6905 - accuracy: 0.532 - ETA: 10s - loss: 0.6914 - accuracy: 0.527 - ETA: 10s - loss: 0.6913 - accuracy: 0.525 - ETA: 10s - loss: 0.6921 - accuracy: 0.517 - ETA: 10s - loss: 0.6917 - accuracy: 0.520 - ETA: 10s - loss: 0.6909 - accuracy: 0.526 - ETA: 10s - loss: 0.6907 - accuracy: 0.529 - ETA: 10s - loss: 0.6910 - accuracy: 0.528 - ETA: 10s - loss: 0.6909 - accuracy: 0.529 - ETA: 10s - loss: 0.6908 - accuracy: 0.529 - ETA: 10s - loss: 0.6904 - accuracy: 0.532 - ETA: 10s - loss: 0.6898 - accuracy: 0.537 - ETA: 10s - loss: 0.6896 - accuracy: 0.538 - ETA: 9s - loss: 0.6898 - accuracy: 0.537 - ETA: 9s - loss: 0.6895 - accuracy: 0.53 - ETA: 9s - loss: 0.6896 - accuracy: 0.53 - ETA: 9s - loss: 0.6895 - accuracy: 0.53 - ETA: 9s - loss: 0.6896 - accuracy: 0.53 - ETA: 9s - loss: 0.6894 - accuracy: 0.53 - 

40268/40268 [==============================] - ETA: 10s - loss: 0.6881 - accuracy: 0.543 - ETA: 10s - loss: 0.6883 - accuracy: 0.554 - ETA: 11s - loss: 0.6904 - accuracy: 0.536 - ETA: 11s - loss: 0.6893 - accuracy: 0.536 - ETA: 11s - loss: 0.6893 - accuracy: 0.536 - ETA: 11s - loss: 0.6889 - accuracy: 0.536 - ETA: 11s - loss: 0.6893 - accuracy: 0.532 - ETA: 11s - loss: 0.6901 - accuracy: 0.528 - ETA: 11s - loss: 0.6912 - accuracy: 0.520 - ETA: 11s - loss: 0.6907 - accuracy: 0.523 - ETA: 11s - loss: 0.6906 - accuracy: 0.525 - ETA: 11s - loss: 0.6907 - accuracy: 0.525 - ETA: 11s - loss: 0.6900 - accuracy: 0.528 - ETA: 11s - loss: 0.6900 - accuracy: 0.529 - ETA: 10s - loss: 0.6907 - accuracy: 0.524 - ETA: 10s - loss: 0.6907 - accuracy: 0.524 - ETA: 10s - loss: 0.6903 - accuracy: 0.527 - ETA: 10s - loss: 0.6900 - accuracy: 0.531 - ETA: 10s - loss: 0.6901 - accuracy: 0.530 - ETA: 10s - loss: 0.6899 - accuracy: 0.531 - ETA: 10s - loss: 0.6897 - accuracy: 0.532 - ETA: 10s - loss: 0.6896 - acc

...

40268/40268 [==============================] - ETA: 12s - loss: 0.6683 - accuracy: 0.562 - ETA: 12s - loss: 0.6525 - accuracy: 0.587 - ETA: 12s - loss: 0.6500 - accuracy: 0.591 - ETA: 13s - loss: 0.6544 - accuracy: 0.593 - ETA: 13s - loss: 0.6562 - accuracy: 0.593 - ETA: 13s - loss: 0.6582 - accuracy: 0.597 - ETA: 12s - loss: 0.6621 - accuracy: 0.590 - ETA: 12s - loss: 0.6621 - accuracy: 0.589 - ETA: 12s - loss: 0.6629 - accuracy: 0.589 - ETA: 12s - loss: 0.6649 - accuracy: 0.588 - ETA: 12s - loss: 0.6623 - accuracy: 0.594 - ETA: 12s - loss: 0.6631 - accuracy: 0.595 - ETA: 12s - loss: 0.6628 - accuracy: 0.596 - ETA: 12s - loss: 0.6636 - accuracy: 0.593 - ETA: 12s - loss: 0.6621 - accuracy: 0.597 - ETA: 12s - loss: 0.6631 - accuracy: 0.594 - ETA: 11s - loss: 0.6626 - accuracy: 0.592 - ETA: 11s - loss: 0.6625 - accuracy: 0.593 - ETA: 11s - loss: 0.6633 - accuracy: 0.592 - ETA: 11s - loss: 0.6632 - accuracy: 0.593 - ETA: 11s - loss: 0.6635 - accuracy: 0.591 - ETA: 11s - loss: 0.6632 - acc

40268/40268 [==============================] - ETA: 11s - loss: 0.6481 - accuracy: 0.605 - ETA: 11s - loss: 0.6590 - accuracy: 0.591 - ETA: 12s - loss: 0.6567 - accuracy: 0.596 - ETA: 11s - loss: 0.6650 - accuracy: 0.585 - ETA: 11s - loss: 0.6635 - accuracy: 0.582 - ETA: 11s - loss: 0.6677 - accuracy: 0.572 - ETA: 11s - loss: 0.6662 - accuracy: 0.574 - ETA: 11s - loss: 0.6655 - accuracy: 0.582 - ETA: 11s - loss: 0.6673 - accuracy: 0.580 - ETA: 11s - loss: 0.6653 - accuracy: 0.580 - ETA: 11s - loss: 0.6654 - accuracy: 0.581 - ETA: 11s - loss: 0.6657 - accuracy: 0.581 - ETA: 11s - loss: 0.6650 - accuracy: 0.580 - ETA: 10s - loss: 0.6634 - accuracy: 0.582 - ETA: 10s - loss: 0.6629 - accuracy: 0.585 - ETA: 10s - loss: 0.6634 - accuracy: 0.584 - ETA: 10s - loss: 0.6624 - accuracy: 0.587 - ETA: 10s - loss: 0.6627 - accuracy: 0.587 - ETA: 10s - loss: 0.6632 - accuracy: 0.587 - ETA: 10s - loss: 0.6610 - accuracy: 0.589 - ETA: 10s - loss: 0.6617 - accuracy: 0.587 - ETA: 10s - loss: 0.6618 - acc

40268/40268 [==============================] - ETA: 11s - loss: 0.6528 - accuracy: 0.589 - ETA: 11s - loss: 0.6640 - accuracy: 0.582 - ETA: 11s - loss: 0.6603 - accuracy: 0.579 - ETA: 11s - loss: 0.6596 - accuracy: 0.581 - ETA: 11s - loss: 0.6632 - accuracy: 0.581 - ETA: 11s - loss: 0.6661 - accuracy: 0.571 - ETA: 11s - loss: 0.6633 - accuracy: 0.573 - ETA: 11s - loss: 0.6635 - accuracy: 0.579 - ETA: 11s - loss: 0.6643 - accuracy: 0.579 - ETA: 11s - loss: 0.6632 - accuracy: 0.579 - ETA: 11s - loss: 0.6620 - accuracy: 0.582 - ETA: 11s - loss: 0.6628 - accuracy: 0.579 - ETA: 11s - loss: 0.6636 - accuracy: 0.576 - ETA: 11s - loss: 0.6633 - accuracy: 0.577 - ETA: 11s - loss: 0.6621 - accuracy: 0.578 - ETA: 11s - loss: 0.6607 - accuracy: 0.580 - ETA: 11s - loss: 0.6620 - accuracy: 0.579 - ETA: 11s - loss: 0.6615 - accuracy: 0.581 - ETA: 11s - loss: 0.6616 - accuracy: 0.582 - ETA: 11s - loss: 0.6618 - accuracy: 0.583 - ETA: 11s - loss: 0.6622 - accuracy: 0.581 - ETA: 11s - loss: 0.6623 - acc

40268/40268 [==============================] - ETA: 11s - loss: 0.6576 - accuracy: 0.570 - ETA: 11s - loss: 0.6709 - accuracy: 0.556 - ETA: 11s - loss: 0.6641 - accuracy: 0.575 - ETA: 12s - loss: 0.6663 - accuracy: 0.566 - ETA: 12s - loss: 0.6625 - accuracy: 0.573 - ETA: 12s - loss: 0.6632 - accuracy: 0.580 - ETA: 12s - loss: 0.6643 - accuracy: 0.576 - ETA: 12s - loss: 0.6659 - accuracy: 0.577 - ETA: 12s - loss: 0.6657 - accuracy: 0.582 - ETA: 12s - loss: 0.6658 - accuracy: 0.583 - ETA: 12s - loss: 0.6664 - accuracy: 0.584 - ETA: 12s - loss: 0.6644 - accuracy: 0.585 - ETA: 12s - loss: 0.6651 - accuracy: 0.584 - ETA: 12s - loss: 0.6632 - accuracy: 0.586 - ETA: 11s - loss: 0.6623 - accuracy: 0.587 - ETA: 11s - loss: 0.6624 - accuracy: 0.585 - ETA: 11s - loss: 0.6635 - accuracy: 0.583 - ETA: 11s - loss: 0.6630 - accuracy: 0.584 - ETA: 11s - loss: 0.6635 - accuracy: 0.584 - ETA: 11s - loss: 0.6633 - accuracy: 0.583 - ETA: 11s - loss: 0.6629 - accuracy: 0.583 - ETA: 11s - loss: 0.6625 - acc

40268/40268 [==============================] - ETA: 11s - loss: 0.6965 - accuracy: 0.492 - ETA: 11s - loss: 0.6756 - accuracy: 0.552 - ETA: 11s - loss: 0.6766 - accuracy: 0.559 - ETA: 11s - loss: 0.6747 - accuracy: 0.564 - ETA: 11s - loss: 0.6720 - accuracy: 0.568 - ETA: 11s - loss: 0.6686 - accuracy: 0.575 - ETA: 11s - loss: 0.6649 - accuracy: 0.578 - ETA: 11s - loss: 0.6623 - accuracy: 0.583 - ETA: 11s - loss: 0.6626 - accuracy: 0.585 - ETA: 11s - loss: 0.6622 - accuracy: 0.587 - ETA: 11s - loss: 0.6626 - accuracy: 0.584 - ETA: 11s - loss: 0.6633 - accuracy: 0.582 - ETA: 11s - loss: 0.6613 - accuracy: 0.584 - ETA: 11s - loss: 0.6588 - accuracy: 0.588 - ETA: 11s - loss: 0.6594 - accuracy: 0.587 - ETA: 11s - loss: 0.6600 - accuracy: 0.587 - ETA: 10s - loss: 0.6610 - accuracy: 0.585 - ETA: 10s - loss: 0.6612 - accuracy: 0.584 - ETA: 10s - loss: 0.6606 - accuracy: 0.585 - ETA: 10s - loss: 0.6602 - accuracy: 0.585 - ETA: 10s - loss: 0.6597 - accuracy: 0.585 - ETA: 10s - loss: 0.6603 - acc

In [25]:
# evaluate the model
loss, accuracy = model.evaluate(reshape_x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy))

Accuracy: 0.515132
time: 1.7 s


In [26]:
%%javascript
Jupyter.notebook.save_checkpoint()

<IPython.core.display.Javascript object>

time: 292 ms


In [27]:
from time import sleep
sleep(60)

time: 1min


In [28]:
from shutil import copyfile
copyfile('RNN_Model_Base.ipynb', folder + '/RNN_Model_Base.ipynb' )

'Resultados/7/RNN_Model_Base.ipynb'

time: 9 ms


In [ ]:
!pip install tensorflow-gpu


In [37]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()

[]

time: 547 ms


### Embedding Suma

In [29]:
# # Selecciono el archivo con el que se corre el modelo
# data = embeddings_sum_individual

time: 146 ms


In [30]:
# # Se separa en train y test
# x_train = data.drop([data.columns[0],"Top","Label", "Date"], axis=1)[:num_training]
# x_test = data.drop([data.columns[0],"Top",'Label', 'Date'], axis=1)[num_training:]
# y_train = data["Label"].values[:num_training]
# y_test = data["Label"].values[num_training:]

time: 115 ms


In [31]:
# x_train_array = x_train.to_numpy()
# reshape_x_train = x_train_array.reshape(40268, 1, 300)
# reshape_x_train.shape

time: 159 ms


In [32]:
# x_test_array = x_test.to_numpy()
# reshape_x_test = x_test_array.reshape(9450, 1, 300)
# reshape_x_test.shape

time: 240 ms


In [33]:
# from numpy.testing import assert_allclose
# from keras.models import Sequential, load_model
# from keras.layers import LSTM, Dropout, Dense
# from keras.callbacks import ModelCheckpoint

time: 208 ms


In [34]:
# n_units = 10

# model = Sequential()
# model.add(LSTM(n_units, input_shape=(1,300), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(n_units, return_sequences=False))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='mean_squared_error', optimizer='adam')
# # compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# # define the checkpoint
# filepath="word2vec-{epoch:02d}-{loss:.4f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

time: 135 ms


In [35]:
# # fit the model
# model.fit(reshape_x_train, y_train, epochs=50, batch_size=500, callbacks=callbacks_list)
# model.summary()

time: 107 ms


In [36]:
# # evaluate the model
# loss, accuracy = model.evaluate(reshape_x_test, y_test, verbose=0)
# print('Accuracy: %f' % (accuracy))

time: 123 ms
